<a href="https://colab.research.google.com/github/N-aksif-N/MineColab_Improved/blob/free-config/MineColabImproved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p>
  <img style="display: block; margin-left: auto; margin-right: auto;" src="https://raw.githubusercontent.com/N-aksif-N/MineColab/master/minecolab.png" alt="" width="170" height="136" />
</p>

<h1 style="text-align: center;">
  <span style="color: #00ffff;">MineColab</span>
</h1>

<hr />
  <h2 style="text-align: center;">
    <span style="color: #99ccff;">Forked By N-Aksif</span>
  </h2>

  <h2 style="text-align: center;">
    <span style="color: #FFFFFF;">Run Minecraft Server On Google Colab</span>
  </h2>
  </span>
  </h2>
  <a href="https://github.com/N-aksif-N/MineColab/tree/main" target="_parent"><img align="right" alt="Open In Github"></a>

>[🔥 Starting](#scrollTo=Ei7_3ODcUdm_)

>[▶ or 🛑  Console](#scrollTo=JMgdsuPcUuEv)

>[⚓ Options](#scrollTo=HD4lBUT9RcS-)

>[📎  Log](#scrollTo=hfUxmQTgUF8Z)

>[📰  Software](#scrollTo=JkGq3KakW-Bc)

>[🎈  Plugins, mods](#scrollTo=QaNRBRuK8seq)

>[📁 File Management](#scrollTo=F7efc3oOqTVQ)



----

In [ ]:
# @title **[❗]  Set up** {display-mode: "form"}

from requests import get
from time import sleep
from json import load, dump
from os import environ
from os.path import exists
try: from jproperties import Properties
except:
  %pip install -q jproperties
  from jproperties import Properties
try: from rich import print
except:
  %pip install -q rich
  from rich import print
if exists('/content/drive') == False:
  from google.colab import drive, runtime
  drive.mount('/content/drive')
else: from google.colab import runtime
try: from pyngrok import conf, ngrok
except:
  %pip install -q pyngrok
  from pyngrok import conf, ngrok
try: from bs4 import BeautifulSoup
except:
  %pip install -q BeautifulSoup4
  from bs4 import BeautifulSoup
drive_path = '/content/drive/MyDrive/minecraft'
%cd '{drive_path}'
#------------------------------------------------------------------------------------------------------------------------------------#
def LOG(*args, sep=''):
  check = False
  args = list(args)
  for i in range(len(args)):
    if isinstance(args[i], str):
      if '\n' in args[i]: args[i] = args[i].replace('\n', ''); args.insert(0, '\n[bold green][ LOG ][/bold green] '); check = True; break
  if check != True: args.insert(0, '[bold green][ LOG ][/bold green] ')
  print(sep.join(map(str, args)))
def COLABCONFIG(server_name):
  return f"{drive_path}/{server_name}/colabconfig.txt"
def COLABCONFIG_LOAD(server_name):
  if exists(COLABCONFIG(server_name)):
    return load(open(COLABCONFIG(server_name)))
  else:
    colabconfig = {"server_type": "generic"} # using default, if config doesn't exist.
    dump(colabconfig, open(COLABCONFIG(server_name),'w'))
    raise Exception('Please checking whether you deleted your colabconfig file or not.')

SERVERCONFIG = f'{drive_path}/server_list.txt'
def SERVER_IN_USE(server_name):
  if exists(f'{drive_path}/{server_name}'):
    return server_name
  else:
    serverconfig = load(open(SERVERCONFIG))
    if serverconfig['server_in_use'] != '':
      return serverconfig['server_in_use']
    else:
      raise Exception('Please create a minecraft server first!')
def GET(url):
  r = get(url)
  if r.status_code == 200:
    return r
  else: raise Exception('Error '+ str(r.status_code) + "! Most likely you entered an unsupported version. Try running the code again if you think that shouldn't have happened.")
def DOWNLOAD_FILE(url, path, file_name):
  # Download file into file_name thourgh url
  r = GET(url)
  LOG('\nDownloading ' + file_name)
  with open(path + '/' + file_name, 'wb') as f:
    f.write(r.content)

#--------------------------------------------------------- INSTALL JAVA --------------------------------------------------------------#
def INSTALL_JAVA(version: str):
  # Adoptopen jdk -- optional
  # !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
  # !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk gets installed."

  # Java 8 is required to run Minecraft versions 1.12 through 1.17. Java 17 is required to run Minecraft version 1.18 and up.
  if version > "1.17":
    ! sudo apt install openjdk-17-jre-headless  > /dev/null &&echo 'Openjdk17 is working correctly, you are good to go.' || "Openjdk doesn't seem to be installed or isn't working, falling back to java 11\nYou might experience reduced performance. Minecraft 1.18 and above might fail to launch."
    environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
    ! update-alternatives --set java /usr/lib/jvm/java-17-openjdk-amd64/bin/java
  else:
    ! sudo apt install openjdk-8-jre-headless  > /dev/null && echo 'Openjdk8 is working correctly, you are good to go.' || echo "Openjdk doesn't seem to be installed or isn't working, falling back to java 11\nYou might experience reduced performance. Minecraft 1.18 and above might fail to launch."
    environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    ! update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
#------------------------------------------------------- INSTALL SERVER.JAR -------------------------------------------------------------#
# The jar file name
def JAR_LIST_RUN(colabconfig):
  return {'generic': 'server.jar', 'vanilla':'vanilla.jar','snapshot': 'snapshot.jar',
          'purpur' : 'purpur.jar', 'paper': 'paper.jar', 'velocity' : 'velocity.jar',
          'fabric' : 'fabric-launcher.jar', 'forge': 'minecraft_server.' + colabconfig['server_version'] + '.jar',
          'arclight' : 'arclight.jar', 'mohist': 'mohist.jar', 'banner': 'banner.jar'}

def SERVERSJAR(all = False, jar = False, server_type= '', version = None):
  # Return the download url (jar) AND return the detail versions for each software (all)
  if all:
    server_branches = []
    if server_type == 'vanilla' or server_type=='snapshot':
      rJSON = GET('https://launchermeta.mojang.com/mc/game/version_manifest.json').json()
      if server_type == 'vanilla': server_type = 'release'
      if version != 'vanilla - latest_version': server_version = [hit["id"] for hit in rJSON["versions"] if hit["type"] == server_type]
      else:
        for hit in rJSON["versions"]:
          if hit["type"] == server_type:
            return hit['id']
    elif server_type == 'paper' or  server_type == 'velocity' or server_type == 'purpur' or server_type == 'mohist' or server_type == 'banner':
      if server_type == 'purpur': rJSON = GET('https://api.purpurmc.org/v2/purpur').json()
      elif server_type == 'mohist' or server_type == 'banner': rJSON = GET(f'https://mohistmc.com/api/v2/projects/{server_type}').json()
      else: rJSON = GET(f'https://api.papermc.io/v2/projects/{server_type}').json()
      if server_type == 'velocity': LOG('Velocity is compatible with many software like forge, fabric, paper, vanilla,...', '\n Before doing anything please look through https://docs.papermc.io/velocity/server-compatibility')
      server_version = [hit for hit in rJSON["versions"]]
    elif server_type == 'fabric':
      rJSON = GET('https://meta.fabricmc.net/v2/versions/game').json()
      server_version = [hit['version'] for hit in rJSON["games"] if hit['stable'] == 'true']
    elif server_type == 'forge':
      rJSON = GET('https://files.minecraftforge.net/net/minecraftforge/forge/index.html')
      soup = BeautifulSoup(rJSON.content, "html.parser")
      server_version = [tag.text for tag in soup.find_all('a') if '.' in tag.text and '\n' not in tag.text]
    else:
      LOG('Before going deeper, please check out https://github.com/IzzelAliz/Arclight')
      rJSON_versions = GET(f'https://files.hypoglycemia.icu/v1/files/arclight/minecraft').json()
      rJSON = GET('https://files.hypoglycemia.icu/v1/files/arclight/branches').json()
      server_version  = [hit['name'] for hit in rJSON["files"]]
      server_branches = [hit['name'] for hit in rJSON["files"] if '.' not in hit['name']]
    return [server_version, server_branches]
  elif jar == True and version != None:
    # RETURN DOWNLOAD URL
    if server_type == 'vanilla' or server_type=='snapshot':
      rJSON = GET('https://launchermeta.mojang.com/mc/game/version_manifest.json').json()
      if server_type == 'vanilla': server_type = 'release'
      for hit in rJSON["versions"]:
        if hit["type"] == server_type and hit['id'] == version:
          return GET(hit['url']).json()["downloads"]['server']['url']
    elif server_type == 'paper' or  server_type == 'velocity':
      build = GET(f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}').json()["builds"][-1]
      jar_name = GET(f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}/builds/{build}').json()["downloads"]["application"]["name"]
      return f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}/builds/{build}/downloads/{jar_name}'
    elif server_type == 'purpur':
      build = GET(f'https://api.purpurmc.org/v2/purpur/{version}').json()["builds"]["latest"]
      return f'https://api.purpurmc.org/v2/purpur/{version}/{build}/download'
    elif server_type == 'mohist' or server_type == 'banner':
      return GET(f'https://mohistmc.com/api/v2/projects/{server_type}/{version}/builds').json()["builds"][-1]["url"]
    elif server_type == 'fabric':
      installerVersion = GET('https://meta.fabricmc.net/v2/versions/installer').json()[0]["version"]
      fabricVersion = GET(f'https://meta.fabricmc.net/v2/versions/loader/{version}').json()[0]["loader"]["version"]
      return "https://meta.fabricmc.net/v2/versions/loader/" + version + "/" + fabricVersion + "/" + installerVersion + "/server/jar"
    elif server_type == 'forge':
      rJSON = GET(f'https://files.minecraftforge.net/net/minecraftforge/forge/index_{version}.html')
      soup = BeautifulSoup(rJSON.content, "html.parser")
      tag =  soup.find('a', title="Installer"); tag = str(tag); tag = tag[tag.find('"') + 1 :]
      link = tag[:tag.find('"')]; link = link[link.find('=') + 1:]; link = link[link.find('=') + 1:]
      return link
    else:
      rJSON = GET(f'https://files.hypertention.cn/v1/files/arclight/branches/{version}/loaders').json()
      LOG('Available type: ')
      print([hit['name'] for hit in rJSON['files']])
      build = input(' Type: ')
      choice = input('Stable(st) or Snapshot(sn): ')
      if choice == 'sn': choice = 'latest-snapshot';
      else: choice = "latest-stable";
      rJSON = GET(f'https://files.hypertention.cn/v1/files/arclight/branches/{version}/loaders/{build}/{choice}').json()
      print([hit['name'] for hit in rJSON['files']])
      _versions_ = input(' Versions: ')
      return f'https://files.hypertention.cn/v1/files/arclight/branches/{version}/loaders/{build}/{choice}/{_versions_}'
  else: raise Exception('Wrong given arguments')
def GET_SERVERJAR(server_name, type_, version, jarname = 'server.jar'):
  # Get jar file
  DOWNLOAD_FILE(url= SERVERSJAR(jar = True, server_type = type_, version= version), path = f"{drive_path}/{server_name}", file_name= jarname)
#------------------------------------------------------ TUNNEL - COMMAND -------------------- ---------------------------------------#
def CONNECT_NGROK(port, type_ , proxy):
  # Get serverconfig['ngrok_proxy'] : dict includes (authtoken, region)
  token = proxy['authtoken']
  !ngrok authtoken $token
  region = proxy['region']
  conf.get_default().region = region
  url = ngrok.connect (port, type_)
  if type_ == 'tcp':
    return 'Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', '')
  else:
    return url
def EXIT_NGROK():
  # Exiting ngrok tunnel helps improve the performance
  tunnels = ngrok.get_tunnels()
  for tunnel in tunnels: ngrok.disconnect(tunnel.public_url)
  ngrok.kill()
  LOG("Ngrok closed.")
def RUNCOMMAND(server_name, colabconfig, serverconfig, hide = False):
  # Get all the improving java arguments
  java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
  args = " -Xms8G -Xmx8G "
  _type = colabconfig["server_type"]

  if _type == "paper" or _type == 'purpur':
    # Improving paper cilent (purpur is an alternative).
    # For more detailed: https://docs.papermc.io/paper/aikars-flags
    args += '-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true '
  elif _type == "velocity":
    args += '-XX:+AlwaysPreTouch -XX:+ParallelRefProcEnabled -XX:+UnlockExperimentalVMOptions -XX:+UseG1GC -XX:G1HeapRegionSize=4M -XX:MaxInlineLevel=15'
  # GC_LOGGING
  if java_ver[0] == "1": args += '-Xloggc:gc.log -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps -XX:+UseGCLogFileRotation -XX:NumberOfGCLogFiles=5 -XX:GCLogFileSize=1M'
  # else: args += '-Xlog:gc*:logs/gc.log:time,uptime:filecount=5,filesize=1M' # GC_Logging may not worked in java 11 and above
  #args += '--add-modules=jdk.incubator.vector' # This argument will be removed in the future

  # Forge need to open and run it before starting server. => Using command instead.
  if _type == 'forge':
    !java -jar forge-installer.jar --installServer
    LOG('\nServer starting...')
  jar_name = JAR_LIST_RUN(colabconfig)[_type]

  if hide == True:
      # Install needed file: eula.txt and so on.
      !java -server $args -jar $jar_name nogui >/dev/null && wait&
  else:
    tunnel_service = serverconfig['ngrok_proxy']['tunnel_service']
    # Starting tunneling and run java file.
    LOG(f'Starting server ({tunnel_service})...')
    LOG('Stop server => /stop')
    if tunnel_service == "ngrok":
      LOG(CONNECT_NGROK(type_= 'tcp', port= 25565, proxy = serverconfig['ngrok_proxy']))
      !java -server $args -jar $jar_name nogui
    elif tunnel_service == "playit":
      # Download playit
      ! command -v playit || curl -SsL https://playit-cloud.github.io/ppa/key.gpg | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/playit.gpg &>/dev/null && echo "deb [signed-by=/etc/apt/trusted.gpg.d/playit.gpg] https://playit-cloud.github.io/ppa/data ./" | sudo tee /etc/apt/sources.list.d/playit-cloud.list && sudo apt -qq update && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "Failed to install playit"
      # Run server
      ! playit & java -server $args -jar $jar_name nogui
    else:
      # Download argo
      if exists(f'{drive_path}/{server_name}/cloudflared-linux-amd64') == False:
        ! curl https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 > cloudflared-linux-amd64
        ! chmod +x cloudflared-linux-amd64
      # Runserver
      !./cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:25565 && echo 'Your free tunnel has started!' && java -server $args -jar $jar_name nogui
#--------------------------------------------------- SERVER - PLUGINS/MODS/MODPACKS SEARCH ----------------------------------------------------------#
def FACETS(colabconfig, software, categories, versions, project_types, proj_types, index):
  # Get all the syntax
  if software == 'Modrinth':
    facets = "["
    if categories != 'none': facets += f'["categories:{[categories]}"]';
    if facets != '[': facets += "," + f'["versions:{[versions]}"]';
    else: facets += f'["versions:{[versions]}"]';
    if facets != '[': facets += "," + f'["project_type:{[proj_types]}"]'
    else: facets += f'["project_type:{[proj_types]}"]'
    facets += "]"; facetsInURL = "";
    if facets != "[]": facetsInURL = f'&facets={facets}&index={index}'
    return facetsInURL
  else:
    if categories != 'none':
      if categories == 'fabric': categories = 4 #Fabric
      elif categories == 'forge': categories = 1 #Forge
      elif categories == 'quilt': categories = 5 # quilt
      categories = f"&modLoaderType={categories}"
    if index != "none":
      if index == "relevance": index = 1 # Featured
      elif index == "downloads": index = 6 #TotalDownloads
      elif index == "follows": index = 2 #Popularity
      elif index == "newest": index = 11 #ReleasedDate
      elif index == "updated": index = 3 #LastUpdated
      index = f"&sortField={index}"
    else: index = ''
    if versions != "none": versions = f"&gameVersion={versions}"
    else: versions = ""
    return categories + versions + index
def SEARCH(software, server_name, search_name, categories, versions, project_types, proj_types, index):
  colabconfig = COLABCONFIG_LOAD(server_name)
  project = {}
  LOG(f'\nSearching for the related of {search_name} ...')
  if software == "Modrinth":
    # Get syntax and get data
    facetsInURL = FACETS(colabconfig, software, categories, versions, project_types, proj_types, index)
    rJSON = GET(f'https://api.modrinth.com/v2/search?query={search_name}{facetsInURL}').json()
    # Get the list of all relevant project
    for hit in rJSON['hits']:
      # Auditing if it for server or not.
      if hit['server_side'] == 'optional' or hit['server_side'] == 'required':
        # Get a full list title : description
        LOG(hit['title'], " : ", hit['description'])
        project[hit['title']] = hit['project_id']
  elif software == 'Curseforge':
    LOG(f"Curseforge doesn't have the exact searching key for cilent-side or server-side => you may get errors when running this {proj_types}")
    facetsInURL = FACETS(colabconfig, software, categories, versions, project_types, proj_types, index)
    # Because I haven't found any corresponded of project types in the search engine of Curseforge, I don't use it for searching.
    # The gameid of Minecraft is 432
    rJSON = GET(f"https://api.curse.tools/v1/cf/mods/search?gameId=432&searchFilter={search_name}{facetsInURL}").json()
    # Get the list of all relevant project
    for hit in rJSON["data"]:
      # Get a full list name: summary
      LOG(hit["name"], ' : ', hit["summary"])
      project[hit['name']] = str(hit["id"])
  # Checking whether your search name wrong or not. If yes => Get the name of project => Get project id
  project_names =''
  if project == {}:
    raise Exception(f"\nSomething went wrong. Please check your search name.")
  else:
    LOG('\nType the project_name you want to download')
    project_names= input('Project_name: ')
    while project_names not in project:
      LOG('\nWrong project_names please type aigain. If you want to quit, type "None".')
      project_names= input('\nProject_name: ')
      if project_names == 'None': break
  return [project, project_names]
def MODPACK(server_name, file_name, software, path):
  !unzip file_name -d file_name.split('.')[0] &>/dev/null&
  if software == 'Modrinth':
    manifest = load(open(f"{path}/modrinth.index.json"))["files"]
    for files in manifest:
      path_ = files["path"].split("/")[0]
      file_name_ = files["path"].split("/")[1]
      DOWNLOAD_FILE(url = files["downloads"],
                    path = f'{drive_path}/{server_name}/{path_}',file_name = file_name_)
  else:
    manifest = load(open(f"{path}/manifest.json"))["files"]
    for files in manifest:
      project_id = files["projectID"]
      file_ID = files["fileID"]
      rJSON = GET(f'https://api.curse.tools/v1/cf/mods/{project_id}/files/{fileId}').json()["data"]
      file_name_ = rJSON["displayName"]
      DOWNLOAD_FILE(url = files["downloads"],
                    path = f'{drive_path}/{server_name}/mods', file_name = file_name_)
def MODRINTH(check, server_name, file_name, software, path, project_id):
  rJSON = GET(f'https://api.modrinth.com/v2/project/{project_id}/version').json()
  for data in rJSON:
    for game_version in data['game_versions']:
      if versions == game_version:
        files = data['files'][0]['filename']; url = files['url']
        DOWNLOAD_FILE(url= url, path = path, file_name= files)
        check = True; break
      if check == True and project_types == 'modpack':
        MODPACK(server_name = server_name, software = software, path = path, file_name= files['filename']); print('\nCompleted.'); break
      elif check: print('\nCompleted.'); break
    if check == False: raise Exception(f"It seems that {software} doesn't support this {project_names}"); break
def CURSEFORGE(check, server_name, file_name, software, path, project_id):
  rJSON = GET(f'https://api.curse.tools/v1/cf/mods/{project_id}').json()['data']['latestFilesIndexes']
  for data in rJSON:
    if versions == data["gameVersion"]:
      files = data['filename'], fileId = data['fileId']; url = f'https://www.curseforge.com/api/v1/mods/{project_id}/files/{fileId}/download'
      DOWNLOAD_FILE(url= url, path = path, file_name= files)
      check = True
    if check == True and project_types == 'modpack':
      MODPACK(server_name = server_name, software = software, path = path, file_name= files['filename']); print('\nCompleted.'); break
    elif check: print('\nCompleted.'); break
  if check == False: raise Exception(f"It seems that {software} doesn't support this {project_names}")
#----------------------------------------------------------- MAIN -------------------------------------------------------------#

def Install_server(tunnel_service , authtoken, region, server_name, type_, version= ''):
  # Auditing whether file is existed.
  if exists(f'{drive_path}/{server_name}') == False:
    # Create folder
    ! mkdir -p "{drive_path}/{server_name}"
    # Get version
    if version == '':
      # Print available version => get version (string variable)
      LOG(f'Latest versions/builds for {type_}')
      list_ = SERVERSJAR(all = True, server_type= type_)
      server_version = list_[0]
      server_branches = list_[1]
      print('\n Aivailable version/builds:\n', server_version)
      if server_branches != []: print(f'branches: {server_branches}')
      version = input('Server versions: ')
      while version == '':
        EROR("Invalid versions")
        print('\n Aivailable version/builds:\n', server_version)
        if server_branches != []: print(f'branches: {server_branches}')
        version = input('Server version: ')
    elif version == 'vanilla - latest_version':
      version = SERVERSJAR(all = True, server_type= type_, version = version)
    # Load serverconfig
    serverconfig = load(open(SERVERCONFIG))
    serverconfig['server_list'] += [server_name]
    serverconfig['server_in_use'] = server_name
    if serverconfig['ngrok_proxy'] == {}:
      if tunnel_service == 'ngrok':
        LOG('Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken')
        authtoken = input('Your authtoken: ')
        LOG('Available Regions:', ' ap - Asia/Pacific (Singapore)', ' au - Australia (Sydney)', ' eu - Europa (Frankfurt - Germany)', ' in - India (Mumbai)', ' jp - Japan (Tokyo)', ' sa - America (São Paulo - Brazil)', ' us - United States (Ohio)', sep='\n')
        region = input('Region: ')
        serverconfig['ngrok_proxy'] = { "tunnel_service" : tunnel_service, 'authtoken' : authtoken, "region" : region}
    dump(serverconfig, open(SERVERCONFIG, 'w'))
    # Set up colabconfig
    colabconfig = {"server_type": type_, "server_version": version}
    dump(colabconfig, open(COLABCONFIG(server_name),'w'))
    # Download jar file
    if type_ == 'forge': jarname = 'forge-installer.jar' # The jar file name (forge need a special process)
    else: jarname = JAR_LIST_RUN(colabconfig)[type_]
    GET_SERVERJAR(server_name, type_, version, jarname = jarname)
    LOG('\nCompleted!')
  else: raise Exception('Lol, you have already installed this server file')

def Delete_server(server_name):
  # Get default server
  serverconfig = load(open(SERVERCONFIG))
  if serverconfig['server_list'] != []:
    if server_name == '': server_name = serverconfig['server_in_use']
    # Auditing whether file is existed.
    if exists(f'{drive_path}/{server_name}'):
      LOG(f'Deleting {server_name}...')
      # Delete folder without noticable
      !rm -rf "{drive_path}/{server_name}" &>/dev/null&
      # Remove the folder name in server config txt files
      serverconfig['server_list'].remove(server_name)
      if serverconfig['server_in_use'] == server_name:
        try: serverconfig['server_in_use']= serverconfig['server_list'][0]
        except: serverconfig['server_in_use'] = ''
      dump(serverconfig, open(SERVERCONFIG, 'w'))
      LOG('Completed')
    else: raise Exception("You haven't installed yet.")
  else: raise Exception("You haven't installed yet.")

def Run_server(server_name):
  # Get default server. Load serverconfig because we will use it later
  serverconfig = load(open(SERVERCONFIG))
  if server_name == '': server_name = serverconfig['server_in_use']
  # Auditing whether file is existed.
  if exists(f'{drive_path}/{server_name}'):
    LOG('Server:  ' + server_name)
    # Cd to server path
    %cd "{drive_path}/{server_name}"
    colabconfig = COLABCONFIG_LOAD(server_name)
    # Install java acording to the version of server.jar
    INSTALL_JAVA(colabconfig['server_version'])
    # Install necessary files
    if exists(f'{drive_path}/{server_name}/eula.txt') == False:
      LOG("\n[Note: Downloading essentials files takes approximately 10-15 minutes. Please wait for a little bit.]")
      RUNCOMMAND(server_name, colabconfig, serverconfig, hide = True)
    # See what changes in the folder
    LOG("\nFiles:")
    !ls -a
    # Accept the eula.txt => Run server.jar
    LOG('\nAccepting Eula.')
    !sed -i 's/eula=false/eula=true/g' eula.txt
    RUNCOMMAND(server_name, colabconfig, serverconfig)
    # Stop minecraft server => Exit tunnel => Cd to home
    LOG("Finalized server.")
    EXIT_NGROK()
    %cd '{drive_path}'
  else: raise Exception("Have you created your server file?")

#--------------------------------------------------- Ulities ----------------------------------------------------------#
def Choose_server():
  serverconfig = load(open(SERVERCONFIG))
  if serverconfig['server_list'] == []: raise Exception(' Creating your minecraft server before choosing')
  else:
    LOG('\nAivailable server:\n')
    for server in serverconfig['server_list']:
      LOG(server)
    serverconfig['server_in_use'] = input('\nYour minecraft server: ')
    dump(serverconfig, open(SERVERCONFIG, 'w'))
    LOG('Completed')

def Custom_props(server_name, server_icon, motd):
  # Default server
  if server_name == '': server_name = SERVER_IN_USE(server_name)
  if exists(f"{drive_path}/{server_name}/server.properties") == False: raise Exception(' Creating your minecraft server before editing properties'
  else:
    LOG('Server:  ' + server_name)
    # Add server icon file
    if server_icon != '': DOWNLOAD_FILE( url = server_icon, path = f'{drive_path}/{server_name}', file_name = 'server_icon.png')
    # Editing MOTD
    if motd != '':
      server_properties = Properties()
      with open(f"{drive_path}/{server_name}/server.properties", "rb") as f:
          server_properties.load(f, "utf-8")
      server_properties["motd"] = motd
      with open(f"{drive_path}/{server_name}/server.properties", "wb") as f:
          server_properties.store(f, encoding="utf-8")
    LOG('Completed')

def Serverproperties(
    server_name, Slots, Gamemode, Difficulty, Cracked, PVP, Command_block, Fly,
    Animals, Monsters, Villagers, Nether, Force_gamemode, Spawn_protection):
  if server_name == '': server_name = SERVER_IN_USE(server_name)
  if exists(f"{drive_path}/{server_name}/server.properties") == False: raise Exception(' Creating your minecraft server before editing properties'
  else:
    server_properties = Properties()
    with open(f"{drive_path}/{server_name}/server.properties", "rb") as f:
        server_properties.load(f, "utf-8")
    # Configuring
    server_properties["max-players"] = str(Slots)
    server_properties["gamemode"] = Gamemode
    server_properties["difficulty"] = Difficulty
    dict_ = {'pvp': PVP, 'enable-command-block': Command_block, 'allow-flight': Fly, 'spawn-animals': Animals, 'spawn-monsters': Monsters,
            'spawn-npcs': Villagers, 'allow-nether': Nether, 'force-gamemode': Force_gamemode, 'spawn-protection': Spawn_protection, "online-mode" : Cracked}
    for keys, value in dict.items(): server_properties[keys] = str(value).lower()
    # Saving
    with open(f"{drive_path}/{server_name}/server.properties", "wb") as f:
        server_properties.store(f, encoding="utf-8")
    LOG('Completed')

def Software(server_name, server_type, version = ''):
  serverconfig = load(open(SERVERCONFIG))
  if server_name == '': server_name = serverconfig['server_in_use']
  Delete_server(server_name)
  sleep(10)
  Install_server(serverconfig['ngrok_proxy']['tunnel_service'] , serverconfig['ngrok_proxy']['authtoken'], serverconfig['ngrok_proxy']['region'], server_name, server_type)

def Install_(server_name, software, url, search_name, categories, versions, project_types, index):
  if server_name == '': server_name = SERVER_IN_USE(server_name)
  LOG(f'Acessing: {server_name}')
  if url != '':
    # Find file_name in download url
    filename = input('File name (optional) : ')
    if filename == '':
      filename = url[url.find("/") + 1:]
      while filename.find("/")!= -1: filename = filename[filename.find("/") + 1:]
    # Download file
    DOWNLOAD_FILE(url= url, path = f'{drive_path}/{server_name}/{proj_types}', file_name= filename)
  else:
    colabconfig = COLABCONFIG_LOAD(server_name)
    if versions == 'default': versions = colabconfig["server_version"]
    if project_types == 'default' :
      if 'fabric' in categories or 'forge' in categories : proj_types = 'mods'
      elif 'paper' in categories or 'purpur' in categories : proj_types = 'plugins'
    elif project_types != 'none': proj_types = project_types
    else: proj_types = ''
    if categories == 'default': categories = colabconfig['server_type']
    a = SEARCH(software, server_name, search_name, categories, versions, project_types, proj_types, index)
    project = a[0]; project_names = a[1]; check = False;
    if project_names != 'None':
      project_id = project[project_names]
      path = f'{drive_path}/{server_name}/{proj_types}'
      if software == 'Modrinth': MODRINTH(check, server_name, file_name, software, path, project_id)
      else: CURSEFORGE(check, server_name, file_name, software, path, project_id)
    else: LOG('Stopping...')

def Backup_files(file_name = '', file_backup = '', server_name= '', server_backup = '', path = '/content/drive/MyDrive/minecraft'):
  # Get the default server
  if server_name == '': server_name = SERVER_IN_USE(server_name)
  # Settings path
  if file_name != '':
    path1 = path + f'/{server_name}/' + file_name
    if server_backup != '':
      if exists(f'{drive_path}/{server_backup}') == False:
        !mkdir '{drive_path}/{server_backup}'
    if file_backup != '' and server_backup != '': path2 = path + f'/{server_backup}/' + file_backup
    elif file_backup != '' and server_backup == '': path2 = path + f'/{server_name}/' + file_backup
    elif file_backup == '' and server_backup != '':  path2 = path + f'/{server_backup}/' + '(back-up)'
    else: path2 = path + f'/{server_name}/' + '(back-up)'
  else:
    path1 = path + f'/{server_name}'
    if server_backup != '': path2 = path + f'/{server_backup}'
    else: path2 = path + f'/{server_name}' + '(back-up)'
  if exists(path2) == False or exists(path1) == False: raise Execptions(' Creating your minecraft server before backing up files')
  else:
    # Zipping
    !zip -r "{path2}".zip path1 &>dev/null && echo "Zipping done!" || echo "Zipping faled."

#------------------------------------------------------------------------------------------------------------------------------------#
!sudo apt update &>/dev/null &&echo 'apt cache successfully updated' || echo "apt cache update failed, you might receive stale packages"
!sudo apt upgrade &>/dev/null &&echo 'apt cache successfully upgraded' || echo "apt cache upgrade failed, you might receive stale packages"
if exists(drive_path) == False:
  !mkdir -p '{drive_path}' &>/dev/null&
if exists(SERVERCONFIG) == False:
  dump({"server_list": [], "server_in_use": "", "ngrok_proxy" : {}}, open(SERVERCONFIG, 'w'))
LOG('Completed')

---

# 🔥 **Starting**
---


In [ ]:
# @markdown ####**Choose your server file name**
server_name = "minecraft" # @param {type:"string"}
# @markdown ####You can either use [ngrok](https://ngrok.com/), [playit.gg](https://playit.gg/) or [cloudflare's argo](https://www.cloudflare.com/). Ngrok is easy to setup and doesn't requires anything to be installed by the clients but it can often be quite unreliable. Argo doesn't have such limitations but requires a bit more work. Playit has provided limited versions but with a static domain.
# Standard turnel service ( Aivailable turnel: ngrok, argo, playit)
tunnel_service = 'ngrok' #@param ['ngrok', 'argo', 'playit']
Install_server(
  #------------------------------#

    tunnel_service = tunnel_service,
    # Set authtoken and region by default
    authtoken = '',
    region = '',

  #------------------------------#
    server_name = server_name, # your server name
    type_ = "vanilla", # server type
    version = 'vanilla - latest_version'
  #------------------------------#
)

In [ ]:
# @markdown ####**Choosing minecraft server**

Choose_server()

In [ ]:
# @markdown ####**Delete server**
Delete_server(server_name = '')

-----------------------------------------------------

# ▶ **or** 🛑  **Console**
---
The main console for your minecraft server

- **[Ngrok](https://ngrok.com)**
  + Follow the prompts.
  + The IP will change whenever you restart the server.
- **[Cloudflare's argo](https://www.cloudflare.com/)** :
    - If the 'Your free tunnel has started!' notification appears => Done.
    - Access to your server:
    1. Download [Cloudflared client](https://github.com/cloudflare/cloudflared/releases/).
    2. Launch the binary with `<your Cloudflare file name> access tcp --hostname <tunnel_address> --url 127.0.0.1:25565` (note: tunnel_address is your address which has been set on your Cloudflare)
    4. Finally, connect to `127.0.0.1:25565` from the minecraft client which is located in that machine.
- **[PlayIt](https://playit.gg/)**: follow the prompts.

In [ ]:
Run_server(
    #------------------------------#
    # Leaving emty string   -> set to default files
    # @markdown
    server_name = ''
    #------------------------------#
)

---

# ⚓ **Options**
---


In [ ]:
# @markdown ###**Server_custom**

# @markdown ####Server icon: **upload image** and get **direct link** from [[ Image ]](https://postimages.org/)
Server_icon = '' # @param {type: 'string'}
# @markdown ####[[ MOTD create **tool** ]](https://mctools.org/motd-creator)
MOTD = '' # @param {type: 'string'}


Custom_props(
    #------------------------------#
    server_name = '', # Customized server file
    #------------------------------#
    server_icon = Server_icon,
    motd = MOTD
    #------------------------------#

)

In [ ]:
# @markdown ###**Server properties**

Slots = 20 # @param {type:"slider", min:0, max:100, step:1}
Gamemode = "survival" # @param ["survival", "creative", "adventure", "spectator"]
Difficulty = "easy" # @param ["peaceful", "easy", "normal", "hard"]
Cracked = True # @param {type:"boolean"}
PVP = True # @param {type:"boolean"}
Command_block = True # @param {type:"boolean"}
Fly = True # @param {type:"boolean"}
Animals = True # @param {type:"boolean"}
Monsters = True # @param {type:"boolean"}
Villagers = True # @param {type:"boolean"}
Nether = True # @param {type:"boolean"}
Force_gamemode = False # @param {type:"boolean"}
Spawn_protection = 16 # @param {type:"slider", min:0, max:100, step:1}



Serverproperties(
    server_name = '',

    Slots,
    Gamemode,
    Difficulty,
    Cracked,
    PVP,
    Command_block,
    Fly,
    Animals,
    Monsters,
    Villagers,
    Nether,
    Force_gamemode,
    Spawn_protection

)

---

# 📎  **Log**
---


In [ ]:
# @markdown **Shows latest log of your minecraft server**
# Getting server_folder

server_name = SERVER_IN_USE()
if os.path.exists(f'/content/drive/My Drive/minecraft/{server_folder}/logs/latest.log'):
  print(f'Server {server_name}')
  with open(f'/content/drive/My Drive/minecraft/{server_folder}/logs/latest.log', 'r') as f:
    print(f.read())
else: EROR("Error: File didn't exists")

---


# 📰  **Software**
---
Change the software (vanilla) to others platforms

---


+ Please checking whether your minecraft server is in Gdrive or not.


In [ ]:

Software(

    # Default server
    server_name = '',

    # Server_version
    version = '', # Customizable version

    # @markdown ####*Choosing server type*
    server_type = "mohist" # @param ["vanilla", "snapshot", "paper", "purpur", "forge", "fabric", 'velocity', 'arclight','mohist', 'banner']

)

---

# 🎈  **Plugins, mods**

####Download modpack/mod/plugin from [curseforge](https://www.curseforge.com/Minecraft) and [modrinth](https://modrinth.com/)
---


In [ ]:
LOG('\nAivailable server:\n')
for server in serverconfig['server_list']:
  LOG(server)

#@markdown ### **Your search name**

search_name = 'Sodium' # @param {type: 'string'}

# @markdown ####Choose the place to download mod/modpack/plugin
software = 'Modrinth' # @param ["Curseforge", "Modrinth"]

# @markdown ###**Details:**
# @markdown
# @markdown #### (none -> don't search (version -- doesn't wellcoming to do this), default -> set up according to colabconfig file)
categories = 'none'   #@param ['none', 'default', 'vanilla', 'fabric', 'forge', 'paper', 'purpur', 'bungeecord', 'velocity', 'waterfall']
versions = "default" # @param ["default"] {allow-input: true}
project_types = 'none' # @param ['none', 'default', 'mod', 'plugin', 'modpack']
index = 'relevance' #@param ['none', 'relevance', 'downloads', 'follows', 'newest', 'updated']

Install_(
    #------------------------------#
    server_name = input('\nServer:'),
    software= software,
    url= '', # Customized urlmodpack (just from curseforge or modpack)
    #------------------------------#
    search_name= search_name,
    categories = categories, versions= versions,
    project_types= project_types, index= index
)

---

# **📁 File Management**
---


In [ ]:
# @markdown Back up server or file?
choice = 'server' # @param ['server', 'file']
file_name = ''; file_backup = ''; server_name= ''; server_backup = ''
if choice == 'server':
  server_name = input('Server name: ')
  server_backup = input('Back up server name: ')
else:
  server_name = input('Server name: ')
  file_name = input('File name')
  file_backup = input('File back up name: ')
Backup_files(
    #------------------------------#
    file_name = file_name,
    file_backup = file_backup,
    server_name= server_name,
    server_backup = server_backup,
    path = '/content/drive/MyDrive/minecraft'
    #------------------------------#
)